In [12]:
# ! pip install slack_sdk

In [11]:
import requests
import datetime
import xml.etree.ElementTree as ET
from slack_sdk import WebClient

import os
SLACK_TOKEN = os.environ["SLACK_TOKEN"]  # get token from environment

# Configuration
URL = "https://calendar.usc.edu/calendar/1.xml"
CHANNEL = "#events-rss"  # or channel ID

client = WebClient(token=SLACK_TOKEN)
today = datetime.date.today()

# Fetch the XML feed
response = requests.get(URL)
response.raise_for_status()
xml_data = response.content
root = ET.fromstring(xml_data)

events_today = []

for item in root.findall(".//item"):
    title = item.findtext("title") or "No title"
    link = item.findtext("link") or ""
    pub_date = item.findtext("{http://purl.org/dc/elements/1.1/}date") or item.findtext("pubDate")

    if pub_date:
        try:
            # Try to parse ISO-style first (<dc:date>)
            if "T" in pub_date:
                event_date = datetime.date.fromisoformat(pub_date.split("T")[0])
            else:
                # Fallback for <pubDate> style
                event_date = datetime.datetime.strptime(pub_date, "%a, %d %b %Y %H:%M:%S %z").date()

            if event_date == today:
                events_today.append(f"{title}\n{link}")
        except Exception as e:
            print("Bad date format:", pub_date, e)

# Post to Slack
if events_today:
    message = "📅 USC events today:\n" + "\n\n".join(f"• {e}" for e in events_today)
    resp = client.chat_postMessage(channel=CHANNEL, text=message)
    print(resp)
else:
    print("No events today.")

{'ok': True, 'channel': 'C09QX1CGREV', 'ts': '1762205911.261819', 'message': {'user': 'U09QJ6KQP98', 'type': 'message', 'ts': '1762205911.261819', 'bot_id': 'B09QX3E10MP', 'app_id': 'A09RCC7JFA4', 'text': ':date: USC events today:\n• Nov 3, 2025: Eco-Monday in Residential Dining at USC Village\n<https://calendar.usc.edu/event/eco-monday-in-residential-dining>\n\n• Nov 3, 2025: Extra Credit Workout Series Powered by Nike\n<https://calendar.usc.edu/event/extra-credit-workout-series-powered-by-nike>\n\n• Nov 3, 2025: Driving Organizational Change with Data\n<https://calendar.usc.edu/event/driving-organizational-change-with-data>\n\n• Nov 3, 2025: Alia Chand: All That Comes After at Roski Studios Building (IFT)\n<https://calendar.usc.edu/event/alia-chand-all-that-comes-after>\n\n• Nov 3, 2025: Katie Jung: Where Shadows Rest at Roski Studios Building (IFT)\n<https://calendar.usc.edu/event/katie-jung-where-shadows-rest>\n\n• Nov 3, 2025: Sarai Mojica: Signifier at Watt Hall (WAH)\n<https://c